In [3]:
import pygame
import numpy as np
import sys

In [4]:
starting_board = [
    ["BR", "BN", "BB", "BQ", "BK", "BB", "BN", "BR"],
    ["BP", "BP", "BP", "BP", "BP", "BP", "BP", "BP"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    ["r", "n", "b", "q", "k", "b", "n", "r"]
]

board = starting_board.copy()

WIDTH, HEIGHT = 512, 512
DIMENSION = 8
SQUARE_SIZE = HEIGHT // DIMENSION

LIGHT_SQUARE = (240, 217, 181)
DARK_SQUARE = (181, 136, 99)

def draw_board(screen):
    for row in range(DIMENSION):
        for col in range(DIMENSION):
            color = LIGHT_SQUARE if (row + col) % 2 == 0 else DARK_SQUARE
            pygame.draw.rect(screen, color, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def place_piece(screen, piece_image_file, row, col):
    img = pygame.image.load(piece_image_file)
    img = pygame.transform.scale(img, (SQUARE_SIZE, SQUARE_SIZE))
    screen.blit(img, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def is_invalid_move(board, start_row, start_col, end_row, end_col, current_player):
    if start_row == end_row and start_col == end_col:
        print("Başlangıç ve bitiş pozisyonları aynı, geçersiz hamle.")
        return True

    # Hedef karede aynı renkten bir taş varsa geçersiz bir hamledir
    if board[end_row][end_col] != ' ' and board[end_row][end_col].islower() == current_player.islower():
        print("Hedef karede aynı renkten bir taş var, geçersiz hamle.")
        return True
    if board[start_row][start_col].lower() != current_player:
        print("Başlangıç karesinde kendi renginizden bir taş yok, geçersiz hamle.")
        return True

    return False

def make_move(board, start_row, start_col, end_row, end_col):
    global en_passant_possible 
    
    if is_valid_castling(board, start_row, start_col, end_row, end_col, board[start_row][start_col]):
        # Küçük rok yapılırsa
        if end_col == 6:
            board[start_row][start_col] = ' '  
            board[start_row][5] = 'K' if board[start_row][start_col].isupper() else 'k'  
            board[start_row][7] = ' ' 
            board[start_row][5] = 'R' if board[start_row][start_col].isupper() else 'r'  
        # Büyük rok yapılırsa
        elif end_col == 2:
            board[start_row][start_col] = ' '  # Kralın başlangıç pozisyonunu boşalt
            board[start_row][3] = 'K' if board[start_row][start_col].isupper() else 'k'  # Kralı yeni pozisyona taşı
            board[start_row][0] = ' '  # Kaleyi yeni pozisyonda boşalt
            board[start_row][3] = 'R' if board[start_row][start_col].isupper() else 'r'  # Kaleyi yeni pozisyona taşı
        return
    if abs(start_row - end_row) == 2 and board[start_row][start_col].lower() == 'p':
        en_passant_possible = True
    else:
        en_passant_possible = False

    board[end_row][end_col] = board[start_row][start_col]
    board[start_row][start_col] = ' '

    if is_invalid_move(board, start_row, start_col, end_row, end_col, board[start_row][start_col]):
        return

    target_piece = board[end_row][end_col]

    if target_piece == ' ':
        board[end_row][end_col] = board[start_row][start_col]
        board[start_row][start_col] = ' '
    elif target_piece.islower() == board[start_row][start_col].islower():
        print("Aynı renkten bir taş var, geçersiz hamle.")
    else:
        board[end_row][end_col] = board[start_row][start_col]
        board[start_row][start_col] = ' '
    
def promote_pawn(board, row, col):
    player = board[row][col].lower()
    if (player == 'w' and row == 0) or (player == 'b' and row == 7):
        print("Piyon tahtanın sonuna ulaştı! Lütfen bir taş seçin: (v, a, k, f)")
        while True:
            piece_type = input().lower()
            if piece_type in ['v', 'a', 'k', 'f']:
                if player == 'w':
                    piece_type = piece_type.upper()
                board[row][col] = piece_type
                break
            else:
                print("Geçersiz taş! Lütfen v, a, k veya f girin.")

def is_valid_pawn_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible):
    # Başlangıç pozisyonundaki taşın oyuncunun taşı olup olmadığını kontrol et
    if board[start_row][start_col].lower() != player:
        return False
    
    # Başlangıç ve bitiş pozisyonları aynı ise geçersiz bir hamle
    if start_row == end_row and start_col == end_col:
        return False
    
    if player == 'w':
        # İlk hamlede iki kare ileriye gidebilir
        if start_row == 6 and end_row == 4 and start_col == end_col and board[5][start_col] == ' ' and board[4][start_col] == ' ':
            return True
        # Diğer durumlarda sadece bir kare ileriye gidebilir
        elif end_row == start_row - 1 and start_col == end_col and board[end_row][end_col] == ' ':
            return True
        # Çaprazda başka bir taş varsa ve o taş farklı renkte ise alabilir
        elif abs(start_col - end_col) == 1 and start_row - end_row == 1 and board[end_row][end_col] != ' ' and board[end_row][end_col].islower():
            return True
        # El pasan hamlesi
        elif en_passant_possible and end_row == 2 and abs(end_col - start_col) == 1 and start_row - end_row == 1 and board[start_row][end_col].lower() == 'p':
            return True
    # Eğer oyuncu siyah ise piyonlar sadece aşağı yönde hareket edebilir
    elif player == 'b':
        # İlk hamlede iki kare ileriye gidebilir
        if start_row == 1 and end_row == 3 and start_col == end_col and board[2][start_col] == ' ' and board[3][start_col] == ' ':
            return True
        # Diğer durumlarda sadece bir kare ileriye gidebilir
        elif end_row == start_row + 1 and start_col == end_col and board[end_row][end_col] == ' ':
            return True
        # Çaprazda başka bir taş varsa ve o taş farklı renkte ise alabilir
        elif abs(start_col - end_col) == 1 and end_row - start_row == 1 and board[end_row][end_col] != ' ' and board[end_row][end_col].isupper():
            return True
        # El pasan hamlesi
        elif en_passant_possible and end_row == 5 and abs(end_col - start_col) == 1 and end_row - start_row == 1 and board[start_row][end_col].lower() == 'p':
            return True
        if player == 'w' and end_row == 0 and board[start_row][start_col].lower() == 'p':
            promote_pawn(board, end_row, end_col)
        elif player == 'b' and end_row == 7 and board[start_row][start_col].lower() == 'p':
            promote_pawn(board, end_row, end_col)
    return False

def is_valid_rook_move(board, start_row, start_col, end_row, end_col, player):

    if board[start_row][start_col].lower() != player:
        return False
    
    if start_row == end_row and start_col == end_col:
        return False
    
    if start_row == end_row or start_col == end_col:

        if start_row == end_row:
            step = 1 if start_col < end_col else -1
            for col in range(start_col + step, end_col, step):
                if board[start_row][col] != ' ':
                    return False
        
        return True
    
    return False

def is_valid_knight_move(start_row, start_col, end_row, end_col):

    row_diff = abs(end_row - start_row)
    col_diff = abs(end_col - start_col)

    if (row_diff, col_diff) in [(1, 2), (2, 1)]:
        return True
    else:
        return False

def is_valid_bishop_move(start_row, start_col, end_row, end_col):

    row_diff = abs(end_row - start_row)
    col_diff = abs(end_col - start_col)

    if row_diff == col_diff:
        return True
    else:
        return False

def is_valid_king_move(start_row, start_col, end_row, end_col):
    row_diff = abs(end_row - start_row)
    col_diff = abs(end_col - start_col)

    if (row_diff <= 1 and col_diff <= 1) and (start_row != end_row or start_col != end_col):
        return True
    else:
        return False

def is_valid_queen_move(start_row, start_col, end_row, end_col):

    row_diff = abs(end_row - start_row)
    col_diff = abs(end_col - start_col)

    if (row_diff == 0 or col_diff == 0 or row_diff == col_diff):
        return True
    else:
        return False
    
def is_square_attacked(board, row, col, player):
    # Belirli bir kareyi tehdit eden herhangi bir taş var mı kontrol edelim

    # Yatay ve dikey kontrol
    for i in range(8):
        # Yatayda aynı sıradaki diğer kareler
        if board[row][i] != ' ' and board[row][i].lower() != player:
            return True
        # Dikeyde aynı sütundaki diğer kareler
        if board[i][col] != ' ' and board[i][col].lower() != player:
            return True

    # Çapraz kontrol
    for i in range(1, 8):
        # Üst sağ çapraz
        if row - i >= 0 and col + i < 8:
            if board[row - i][col + i] != ' ' and board[row - i][col + i].lower() != player:
                return True
        # Üst sol çapraz
        if row - i >= 0 and col - i >= 0:
            if board[row - i][col - i] != ' ' and board[row - i][col - i].lower() != player:
                return True
        # Alt sağ çapraz
        if row + i < 8 and col + i < 8:
            if board[row + i][col + i] != ' ' and board[row + i][col + i].lower() != player:
                return True
        # Alt sol çapraz
        if row + i < 8 and col - i >= 0:
            if board[row + i][col - i] != ' ' and board[row + i][col - i].lower() != player:
                return True

    # At kontrol
    knight_moves = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (-1, 2), (1, -2), (-1, -2)]
    for dr, dc in knight_moves:
        if 0 <= row + dr < 8 and 0 <= col + dc < 8:
            if board[row + dr][col + dc].lower() == 'n' and board[row + dr][col + dc].islower() != player.islower():
                return True

    # Kale kontrol
    rook_directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for dr, dc in rook_directions:
        r, c = row, col
        while 0 <= r + dr < 8 and 0 <= c + dc < 8:
            r += dr
            c += dc
            if board[r][c] != ' ':
                if board[r][c].lower() == 'r' and board[r][c].islower() != player.islower():
                    return True
                else:
                    break

    # Fil kontrol
    bishop_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    for dr, dc in bishop_directions:
        r, c = row, col
        while 0 <= r + dr < 8 and 0 <= c + dc < 8:
            r += dr
            c += dc
            if board[r][c] != ' ':
                if board[r][c].lower() == 'b' and board[r][c].islower() != player.islower():
                    return True
                else:
                    break

    # Şah kontrol
    king_moves = [(1, 1), (1, 0), (1, -1), (0, 1), (0, -1), (-1, 1), (-1, 0), (-1, -1)]
    for dr, dc in king_moves:
        if 0 <= row + dr < 8 and 0 <= col + dc < 8:
            if board[row + dr][col + dc].lower() == 'k' and board[row + dr][col + dc].islower() != player.islower():
                return True

    return False

def is_valid_castling(board, start_row, start_col, end_row, end_col, player):
    if board[start_row][start_col].lower() != 'k':
        return False
    
 
    if end_row != start_row:
        return False
    
    if abs(end_col - start_col) != 2:
        return False
    
    if player == 'w':
        row = 7
    else:
        row = 0 
    
    # Küçük rok
    if end_col == 6:
        if board[row][5] != ' ' or board[row][6] != ' ':  
            return False
        if is_square_attacked(board, row, 4, player) or is_square_attacked(board, row, 5, player): # Burada taşların kontrolü yapılıyor
            return False
        else:
            return True # Küçük rok geçerli
    # Büyük rok
    elif end_col == 2:
        if board[row][1] != ' ' or board[row][2] != ' ' or board[row][3] != ' ':  # Boş kareler
            return False
        if is_square_attacked(board, row, 2, player) or is_square_attacked(board, row, 3, player):
            return False
        else:
            return True # Büyük rok geçerli
    
    return False # Rok geçersiz

def is_valid_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible):
    if is_valid_castling(board, start_row, start_col, end_row, end_col, player):
        return True
    piece = board[start_row][start_col]

    if board[end_row][end_col].lower() == board[start_row][start_col].lower():
        return False

    if piece.lower() == 'p':
        return is_valid_pawn_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible)
    elif piece.lower() == 'r':
        return is_valid_rook_move(board, start_row, start_col, end_row, end_col, player)
    elif piece.lower() == 'n':
        return is_valid_knight_move(start_row, start_col, end_row, end_col)
    elif piece.lower() == 'b':
        return is_valid_bishop_move(start_row, start_col, end_row, end_col)
    elif piece.lower() == 'q':
        return is_valid_queen_move(start_row, start_col, end_row, end_col)
    elif piece.lower() == 'k':
        return is_valid_king_move(start_row, start_col, end_row, end_col)

    return False

def get_clicked_square(mouse_pos):
    col = mouse_pos[0] // SQUARE_SIZE
    row = (DIMENSION - 1) - (mouse_pos[1] // SQUARE_SIZE) 
    return row, col

def main():
    current_player = 'w'
    
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Chess")

    clock = pygame.time.Clock()
    screen.fill((255, 255, 255))
    en_passant_possible = False 
    running = True
    start_square = None
    end_square = None

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:  # Sol tıklama
                mouse_pos = pygame.mouse.get_pos()
                row, col = get_clicked_square(mouse_pos)
                if start_square is None:
                    if board[row][col].lower() == current_player:
                        start_square = (row, col)
                else:
                    end_square = (row, col)
                    if start_square is not None and end_square is not None:  # None değilse hamle yap
                        if is_valid_move(board, start_square[0], start_square[1], end_square[0], end_square[1], current_player, en_passant_possible):
                            make_move(board, start_square[0], start_square[1], end_square[0], end_square[1])
                            current_player = 'w' if current_player == 'b' else 'b'
        # Hamle yapıldıktan sonra start_square ve end_square'ı sıfırla
                    start_square = None
                    end_square = None

        screen.fill((255, 255, 255)) 

        draw_board(screen)  
        for row in range(DIMENSION):
            for col in range(DIMENSION):
                piece = board[row][col]
                if piece != ' ':
                    piece_image_file = piece.upper() + ".png"
                    place_piece(screen, piece_image_file, row, col)
        pygame.display.flip()

        if start_square is not None:
            pygame.draw.rect(screen, (0, 255, 0), (start_square[1] * SQUARE_SIZE, (DIMENSION - 1 - start_square[0]) * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 4)

        if end_square is not None:
            pygame.draw.rect(screen, (255, 0, 0), (end_square[1] * SQUARE_SIZE, (DIMENSION - 1 - end_square[0]) * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 4)

        pygame.display.flip()  
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()
    

KeyboardInterrupt: 